In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
df_order = pd.read_csv("./Data/order_total.csv")

In [3]:
df_order.head(5)

,Unnamed: 0,ORDER_ID,USER_ID,PRODUCT_ID,ORDER_QUANTITY,ORDER_DT,PAY_TYPE,PAY_NAME,USER_NAME,NICKNAME,GENDER,BIRTHDAY,PHONE_NUMBER,EMAIL,ADDRESS,ITEM_NAME,CATEGORY_ID,PRICE
0,0,Z0000000001,happygirl,A00000002,1,2022-09-16,P01,CARD,한소진,소진,F,1994-08-29,010-9999-9999,efefe@naver.com,서울특별시 관악구,하의,C001,60000
1,1,Z0000000002,nari555,A00000023,5,2022-09-16,P03,KAKAOPAY,김나리,국악소녀,F,1997-05-05,010-8888-1234,nari22@naver.com,경기도 이천시,라면,C004,1200
2,2,Z0000000003,nari555,A00000029,1,2022-09-16,P01,CARD,김나리,국악소녀,F,1997-05-05,010-8888-1234,nari22@naver.com,경기도 이천시,소세지,C004,4500
3,3,Z0000000004,jmlove1234,A00000004,1,2022-09-16,P01,CARD,이강주,참나,F,1989-03-22,010-3234-1242,kjoo@google.com,대전광역시 유성구,가방,C001,100000
4,4,Z0000000005,young876,A00000022,3,2022-09-16,P01,CARD,박가영,이거먹엉,F,1989-08-16,010-2345-1234,young1234@naver.com,광주광역시 동구,과자,C004,1500


In [4]:
df_order['PAY_NAME'].unique()

array(['CARD', 'KAKAOPAY', 'NAVERPAY', 'PAYCO'], dtype=object)

In [5]:
df_order['TOTAL_PRICE'] = df_order['ORDER_QUANTITY'] * df_order['PRICE']

In [6]:
df_c = df_order[df_order['PAY_NAME'] == 'CARD']
df_k = df_order[df_order['PAY_NAME'] == 'KAKAOPAY']
df_n = df_order[df_order['PAY_NAME'] == 'NAVERPAY']
df_p = df_order[df_order['PAY_NAME'] == 'PAYCO']

In [7]:
df_c_price = df_c['TOTAL_PRICE']
df_k_price = df_k['TOTAL_PRICE']
df_n_price = df_n['TOTAL_PRICE']
df_p_price = df_p['TOTAL_PRICE']

## 두 집단 평균 차이 검정 : z-value

* H0 : 카드 사용자 평균 판매 금액과 카카오페이 사용자 평균 판매 금액의 차이가 없다.
* H1 : 카드 사용자 평균 판매 금액이 카카오페이 사용자 평균 판매 금액보다 크다.

In [8]:
df_c_mean = np.mean(df_c_price)
print(df_c_mean)

222872.0588235294


In [9]:
df_c_var = np.std(df_c_price)**2
print(df_c_var)

224973349072.2318


In [10]:
n1 = len(df_c_price)
print(n1)

68


In [11]:
df_k_mean = np.mean(df_k_price)
df_k_var = np.std(df_k_price)**2
n2 = len(df_k_price)

In [12]:
s = ((df_c_var/n1) + (df_k_var/n2))**0.5
print(s)

72213.03671352251


In [13]:
z_value = (df_c_mean - df_k_mean) / s
print(z_value)

1.0058447284240675


In [14]:
z_dist = stats.norm(0, 1)
p_value = 1 - z_dist.cdf(z_value)
# 이 때, cdf는 누적 분포 함수를 의미함

print(p_value)

0.1572451336994194


## ANOVA 검정

In [21]:
# 정규성 검정
test_stat, p = stats.shapiro(df_c_price)
print("검정통계량 : {}, p-value : {}".format(test_stat, p))

test_stat, p = stats.shapiro(df_k_price)
print("검정통계량 : {}, p-value : {}".format(test_stat, p))

test_stat, p = stats.shapiro(df_n_price)
print("검정통계량 : {}, p-value : {}".format(test_stat, p))

test_stat, p = stats.shapiro(df_p_price)
print("검정통계량 : {}, p-value : {}".format(test_stat, p))

# H0 : 정규 분포를 따른다 vs. H1 : 정규 분포를 따르지 않는다
# 정규 분포를 따르지 않음

검정통계량 : 0.5087577104568481, p-value : 9.8985483167318e-14
검정통계량 : 0.3661268353462219, p-value : 4.495038414956061e-18
검정통계량 : 0.588603138923645, p-value : 1.720039215225011e-11
검정통계량 : 0.5554932355880737, p-value : 4.5594740072374407e-07


In [22]:
# 분산 동질성 검정
stats.bartlett(df_c_price, df_k_price, df_n_price, df_p_price)

# H0 : 분산이 동일하다 vs. H1 : 분산이 동일하지 않다
# 분산이 동일하지 않음

BartlettResult(statistic=7.960811549511178, pvalue=0.046828593933715426)

In [23]:
f_val, p_val = stats.f_oneway(df_c_price, df_k_price, df_p_price)
print('F-통계량 : ', f_val)
print('p-value : ', p_val)

F-통계량 :  0.5472792305647792
p-value :  0.579486591257784


In [25]:
f_val, p_val = stats.f_oneway(df_c_price, df_k_price, df_n_price, df_p_price)
print('F-통계량 : ', f_val)
print('p-value : ', p_val)

# H0 : 그룹의 평균 판매 금액 차이가 없다 vs. H1 : 그룹의 평군 판매 금액에 차이가 있다
# 검정 결과 차이가 없음

F-통계량 :  1.2582892459635282
p-value :  0.28941170124619253


## 분할표를 활용한 연관성 분석 : Chi-square

In [27]:
df_order['CARD_OR_PAY'] = np.where(df_order['PAY_NAME']=='CARD', 'CARD', 'PAY')

In [28]:
df_order.head(5)

,Unnamed: 0,ORDER_ID,USER_ID,PRODUCT_ID,ORDER_QUANTITY,ORDER_DT,PAY_TYPE,PAY_NAME,USER_NAME,NICKNAME,GENDER,BIRTHDAY,PHONE_NUMBER,EMAIL,ADDRESS,ITEM_NAME,CATEGORY_ID,PRICE,TOTAL_PRICE,CARD_OR_PAY
0,0,Z0000000001,happygirl,A00000002,1,2022-09-16,P01,CARD,한소진,소진,F,1994-08-29,010-9999-9999,efefe@naver.com,서울특별시 관악구,하의,C001,60000,60000,CARD
1,1,Z0000000002,nari555,A00000023,5,2022-09-16,P03,KAKAOPAY,김나리,국악소녀,F,1997-05-05,010-8888-1234,nari22@naver.com,경기도 이천시,라면,C004,1200,6000,PAY
2,2,Z0000000003,nari555,A00000029,1,2022-09-16,P01,CARD,김나리,국악소녀,F,1997-05-05,010-8888-1234,nari22@naver.com,경기도 이천시,소세지,C004,4500,4500,CARD
3,3,Z0000000004,jmlove1234,A00000004,1,2022-09-16,P01,CARD,이강주,참나,F,1989-03-22,010-3234-1242,kjoo@google.com,대전광역시 유성구,가방,C001,100000,100000,CARD
4,4,Z0000000005,young876,A00000022,3,2022-09-16,P01,CARD,박가영,이거먹엉,F,1989-08-16,010-2345-1234,young1234@naver.com,광주광역시 동구,과자,C004,1500,4500,CARD


In [29]:
df_order['GENDER'].unique()

array(['F', 'M'], dtype=object)

In [30]:
df_order['CARD_OR_PAY'].unique()

array(['CARD', 'PAY'], dtype=object)

In [33]:
female = df_order['GENDER'] == 'F'
male =  df_order['GENDER'] == 'M'
card = df_order['CARD_OR_PAY'] == 'CARD'
pay = df_order['CARD_OR_PAY'] == 'PAY'

In [36]:
f_c = len(df_order[female & card])
f_p = len(df_order[female & pay])
m_c = len(df_order[male & card])
m_p = len(df_order[male & pay])

In [38]:
obs = np.array([[f_c, f_p], [m_c, m_p]])
chi2, p, d, expected = stats.chi2_contingency(obs)

msg = 'Test Statistic : {}\np-value: {}\nDegree of Freedom : {}'
print(msg.format(chi2, p, d))
print(expected)

# H0 : 연관성이 없다 vs. H1 : 연관성이 있다
# 검정 결과 성별과 카드 사용 여부는 연관성이 없음

Test Statistic : 0.8839313900873409
p-value: 0.34712715957397844
Degree of Freedom : 1
[[ 49.58333333 125.41666667]
 [ 18.41666667  46.58333333]]
